# H3 Visualization

Uber provides an open source repository [h3-py-notebooks](https://github.com/uber/h3-py-notebooks) with tutorials, examples and visualizations of their H3 geospatial indexing system.

The code below originally comes from [one of the example notebooks](https://github.com/uber/h3-py-notebooks/blob/master/notebooks/usage.ipynb). The method was, however, modified to take weight as an additional parameter. The weight determines the thickness of lines defining hexagons plotted on the map.

In [16]:
from h3 import h3
import folium
import plotly.express as px
from shapely.geometry import Polygon
from geojson import Feature, FeatureCollection

In [17]:
def visualize_hexagons(hexagons, color="red", weight=8, folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    f = folium.Figure(width=700, height=500)

    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=11, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=weight,color=color)
        m.add_child(my_PolyLine)

    f.add_child(m)
    return f

Following methods originate from this [medium article](https://medium.com/analytics-vidhya/how-to-create-a-choropleth-map-using-uber-h3-plotly-python-458f51593548), which is a helpful tutorial on how to use Uber H3 and Plotly.

In [18]:
def add_geometry(row):
    points = h3.h3_to_geo_boundary(row.hex, True)
    return Polygon(points)

In [19]:
def hexagons_dataframe_to_geojson(df_hex, value_field, file_output = None):

    list_features = []

    for i, row in df_hex.iterrows():
        feature = Feature(geometry = row['geometry'],
                          id = row['hex'],
                          properties = {"value": row[value_field]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    if file_output is not None:
        with open(file_output, "w") as f:
            json.dump(feat_collection, f)

    else :
      return feat_collection

In [20]:
def plot_frequency(dataset, variable, labels, range_color, palette="RdBu"):
    geojson_obj = (hexagons_dataframe_to_geojson(dataset, value_field=variable))

    fig = (px.choropleth_mapbox(
                    dataset,
                    width=700,
                    height=500,
                    geojson=geojson_obj, 
                    locations='hex', 
                    color=variable,
                    color_continuous_scale=palette,
                    range_color=range_color,
                    mapbox_style='carto-positron',
                    zoom=10,
                    center = {"lat": 34.052235, "lon": -118.4},
                    opacity=0.7,
                    labels=labels))
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig 